In [43]:
import seaborn as sns
# ^^^ pyforest auto-imports - don't write above this line
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D, Flatten, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [9]:
(trainx,  trainy) , (testx, testy) = mnist.load_data()

In [10]:
trainx = trainx.reshape((60000, 28, 28, 1))
trainx = trainx.astype('float32') / 255

testx = testx.reshape((10000, 28, 28, 1))
testx = testx.astype('float32') / 255

trainy = to_categorical(trainy)
testy = to_categorical(testy)

In [11]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape= (28, 28, 1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)               

In [12]:
history = model.fit(trainx, trainy, epochs=6, batch_size=64)

Epoch 1/6
60000/60000 [==============================] - 24s 406us/step - loss: 0.1748 - accuracy: 0.9448
Epoch 2/6
60000/60000 [==============================] - 28s 465us/step - loss: 0.0466 - accuracy: 0.9857
Epoch 3/6
60000/60000 [==============================] - 28s 462us/step - loss: 0.0326 - accuracy: 0.9903
Epoch 4/6
60000/60000 [==============================] - 28s 469us/step - loss: 0.0252 - accuracy: 0.9928
Epoch 5/6
60000/60000 [==============================] - 28s 474us/step - loss: 0.0186 - accuracy: 0.9944
Epoch 6/6
60000/60000 [==============================] - 29s 481us/step - loss: 0.0161 - accuracy: 0.9950


In [13]:
test_loss, test_accuracy = model.evaluate(testx, testy)

10000/10000 [==============================] - 1s 107us/step


In [14]:
test_accuracy

0.9919999837875366

In [1]:
#Kaggle Comptetion for MNIST

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.optimizers import Adam ,RMSprop
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.preprocessing.image import ImageDataGenerator

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

Using TensorFlow backend.


In [63]:
train = pd.read_csv("datasets/mnist/train.csv")
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
test= pd.read_csv("datasets/mnist/test.csv")
print(test.shape)
test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
Y_train = train["label"]
# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

In [68]:
X_train = X_train / 255.0
test = test / 255.0

In [69]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [81]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [82]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [84]:
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape= (28, 28, 1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_12 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)              

In [54]:
history = model.fit(X_train, Y_train, epochs=6, batch_size=64)

Epoch 1/6
37800/37800 [==============================] - 14s 378us/step - loss: 0.2371 - accuracy: 0.9261
Epoch 2/6
37800/37800 [==============================] - 16s 430us/step - loss: 0.0603 - accuracy: 0.9813
Epoch 3/6
37800/37800 [==============================] - 17s 460us/step - loss: 0.0403 - accuracy: 0.9879
Epoch 4/6
37800/37800 [==============================] - 17s 463us/step - loss: 0.0298 - accuracy: 0.9906
Epoch 5/6
37800/37800 [==============================] - 18s 468us/step - loss: 0.0228 - accuracy: 0.9935
Epoch 6/6
37800/37800 [==============================] - 18s 478us/step - loss: 0.0176 - accuracy: 0.9945s - - ETA: 0s - loss: 0.0176 - 


In [57]:
test_loss, test_accuracy = model.evaluate(X_val, Y_val)

4200/4200 [==============================] - 0s 110us/step


In [58]:
test_accuracy

0.9900000095367432

In [59]:
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [60]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)